Reading file

In [11]:
import numpy as np
import pandas as pd

In [12]:
taste = pd.read_csv('taste_profiles_V5.csv')
taste.head()

,Unnamed: 0,Product_description,Key_ingredient,Food_group,m_sweet,m_sour,m_bitter,m_umami,m_salt,m_fat
0,0,Potatoes mashed prep w semi-sk milk+marg,mashed potato,Potatoes,7,3,1,15,44,36
1,1,Rosti prepared without fat,rosti,Potatoes,9,3,1,15,35,29
2,2,Chips pre-fried unprepared,chips,Potatoes,9,3,0,7,15,42
3,3,Potatoes boiled with skin av,potato,Potatoes,6,2,1,8,6,9
4,4,Potato waffels/balls frozen unprepared,potato ball,Potatoes,7,3,1,13,40,45


In [13]:
rp = pd.read_csv('first_tenth_recipes.csv')
rp.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [14]:
rp.rename(columns={'Unnamed: 0':'recipeID'}, inplace=True)
rp.head()

,recipeID,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [15]:
rping = pd.read_csv('recipe_ingredientsv3.csv')
rping.head()

,recipeID,quantity,unit,ingredient_NER,ingredient_Key_ingredient,rest of ingredient
0,0,0.500000,cups,milk,milk (low fat),evaporated milk
1,0,0.500000,teaspoons,vanilla,vanilla pudding,vanilla
2,0,2.000000,tablespoons,butter,salted butter,butter or margarine
3,2,0.333333,cups,butter,salted butter,"butter, cubed"
4,2,0.250000,teaspoons,pepper,red bell pepper,pepper


RDF  
Step 1) Finding all triples: subject, predicate, object  
Step 2) Making a predicate: recipeA is a satisfying substitute for recipeB.   
                            Satisfying nutritions + flavors  
Step 3) Visualizing the graph  

```
Questions
namespace example? 
RDF scheme?
Product_description is to be deleted? (Yes)
BNode vs URIRef? (BNode probably)

taste
    Product_description hasKeyIngredient Key_ingredient 
    Key_ingredient hasSweetness m_sweet
    Key_ingredient hasSourness m_sour
    Key_ingredient hasBitterness m_bitter
    Key_ingredient hasUmaminess m_umami
    Key_ingredient hasSaltness m_salf
    Key_ingredient hasfatness m_fat
    (Product_description hasSweetness m_sweet)
    (Automatic...)
    Key_ingredient hasCarb carb
    Key_ingredient hasProtein protein
    Key_ingredient hasFat fat

recipe + recipe_ingredients
    title hasID Unnamed:0 (Vise versa?)
    recipeID is Unnamed:0 (change Unnamed:0 to recipeID)
    title hasNERIngredient ingredient_NER (Not needed?)
    title hasKeyIngredient ingredient_Key_ingredient
    (rest of ingredient not necessary?)
    ingredient_NER hasQuantity quantity
    ingredient_Key_ingredient hasQuantity quantity
    ingredient_NER hasUnit unit
    ingredient_Key_ingredient hasQuantity quantity
    # How to deal with specific recipe has diff quantity, unit?
    # define property to object or include property in object 
    recipe_ID hasIngredient ingredient_Key_ingredient

taste + recipe + recipe_ingredients
    Product's key ingredient is the ingredient 

```

In [6]:
from rdflib import Graph, Literal, BNode, Namespace, RDF

n = Namespace("http://example.org/")
g = Graph()

for index, row in taste.iterrows():
    taste = BNode()
    g.add((taste, RDF.type, n.TasteProfile))
    g.add((taste, n.product, Literal(row['Product_description'])))
    g.add((taste, n.key_ingredient, Literal(row['Key_ingredient'])))
    g.add((taste, n.sweetness, Literal(row['m_sweet'])))
    g.add((taste, n.sourness, Literal(row['m_sour'])))
    g.add((taste, n.bitterness, Literal(row['m_bitter'])))
    g.add((taste, n.umaminess, Literal(row['m_umami'])))
    g.add((taste, n.saltness, Literal(row['m_salt'])))
    g.add((taste, n.fatness, Literal(row['m_fat'])))
    # g.add((ingredient, n.carb, Literal(row['carb'])))
    # g.add((ingredient, n.protein, Literal(row['protein'])))
    # g.add((ingredient, n.fat, Literal(row['fat'])))

for index, row in rp.iterrows():
    recipe = BNode()
    g.add((recipe, RDF.type, n.Recipe))
    g.add((recipe, n.recipe_name, Literal(row['title'])))
    g.add((recipe, n.recipeID, Literal(row['recipeID'])))

for index, row in rping.iterrows():
    ingredient = BNode()
    g.add((ingredient, RDF.type, n.Ingredient))
    g.add((ingredient, n.recipeID, Literal(row['recipeID'])))
    g.add((ingredient, n.quantity, Literal(row['quantity'])))
    g.add((ingredient, n.unit, Literal(row['unit'])))
    g.add((ingredient, n.ingredient_Key_ingredient, Literal(row['ingredient_Key_ingredient'])))

print(g.serialize(format='turtle'))


KeyboardInterrupt: 

In [16]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, RDFS, XSD, URIRef
import re

n = Namespace("http://example.org/")
recipe_ns = Namespace("http://example.org/recipe/")
ingredient_ns = Namespace("http://example.org/ingredient/")
product_ns = Namespace("http://example.org/product/")

g = Graph()

# define classes and properties
g.add((n.Recipe, RDF.type, RDFS.Class))
g.add((n.Ingredient, RDF.type, RDFS.Class))
g.add((n.hasIngredient, RDF.type, RDF.Property))
g.add((n.hasIngredient, RDFS.domain, n.Recipe))
g.add((n.hasIngredient, RDFS.range, n.Ingredient))

# define quantity and unit properties
g.add((n.hasQuantityOf, RDF.type, RDF.Property))
g.add((n.hasQuantityOf, RDFS.domain, n.Ingredient))
g.add((n.hasQuantityOf, RDFS.range, XSD.integer))
g.add((n.hasUnit, RDF.type, RDF.Property))
g.add((n.hasUnit, RDFS.domain, n.Ingredient))
g.add((n.hasUnit, RDFS.range, XSD.string))

# define flavor properties
g.add((n.hasSweetness, RDF.type, RDF.Property))
g.add((n.hasSweetness, RDFS.domain, n.Ingredient))
g.add((n.hasSweetness, RDFS.range, XSD.integer))
g.add((n.hasSourness, RDF.type, RDF.Property))
g.add((n.hasSourness, RDFS.domain, n.Ingredient))
g.add((n.hasSourness, RDFS.range, XSD.integer))
g.add((n.hasBitterness, RDF.type, RDF.Property))
g.add((n.hasBitterness, RDFS.domain, n.Ingredient))
g.add((n.hasBitterness, RDFS.range, XSD.integer))
g.add((n.hasUmaminess, RDF.type, RDF.Property))
g.add((n.hasUmaminess, RDFS.domain, n.Ingredient))
g.add((n.hasUmaminess, RDFS.range, XSD.integer))
g.add((n.hasSaltness, RDF.type, RDF.Property))
g.add((n.hasSaltness, RDFS.domain, n.Ingredient))
g.add((n.hasSaltness, RDFS.range, XSD.integer))
g.add((n.hasFatness, RDF.type, RDF.Property))
g.add((n.hasFatness, RDFS.domain, n.Ingredient))
g.add((n.hasFatness, RDFS.range, XSD.integer))

# define nutrition properties
g.add((n.hasCarb, RDF.type, RDF.Property))
g.add((n.hasCarb, RDFS.domain, n.Ingredient))
g.add((n.hasCarb, RDFS.range, XSD.integer))
g.add((n.hasProtein, RDF.type, RDF.Property))
g.add((n.hasProtein, RDFS.domain, n.Ingredient))
g.add((n.hasProtein, RDFS.range, XSD.integer))
g.add((n.hasFat, RDF.type, RDF.Property))
g.add((n.hasFat, RDFS.domain, n.Ingredient))
g.add((n.hasFat, RDFS.range, XSD.integer))

# recipe has recipeID
g.add((n.hasRecipeID, RDF.type, RDF.Property))
g.add((n.hasRecipeID, RDFS.domain, n.Recipe))
g.add((n.hasRecipeID, RDFS.range, XSD.integer))

# in case we use Product_describtion's key ingredient is the ingredient 
g.add((n.Product, RDF.type, RDFS.Class))
g.add((n.hasKeyIngredient, RDF.type, RDF.Property))
g.add((n.hasKeyIngredient, RDFS.domain, n.Product))
g.add((n.hasKeyIngredient, RDFS.range, n.Ingredient))

<Graph identifier=N6bd5afe5dff74fe5be41eab6843b2475 (<class 'rdflib.graph.Graph'>)>

In [26]:
def new_uri(uri):
    cleaned_uri = re.sub('[^a-zA-Z0-9]', '_', uri)
    return cleaned_uri.replace(' ', '_').replace('.', '_')

for index, row in taste.iterrows():
    product = URIRef(product_ns + new_uri(str(row['Product_description'])))
    ingredient = URIRef(ingredient_ns + new_uri(str(row['Key_ingredient'])))
    g.add((product, n.hasKeyIngredient, ingredient))
    g.add((ingredient, n.hasSweetness, Literal(row['m_sweet'], datatype=XSD.integer)))
    # g.add((ingredient, n.hasFatness, Literal(row['m_fat'], datatype=XSD.integer)))

for index, row in rp.iterrows():
    recipe = URIRef(recipe_ns + new_uri(str(row['recipeID'])))
    g.add((recipe, n.hasRecipeID, Literal(row['recipeID'], datatype=XSD.integer)))

for index, row in rping.iterrows():
    recipe = URIRef(recipe_ns + new_uri(str(row['recipeID'])))
    ingredient = URIRef(ingredient_ns + new_uri(str(row['ingredient_Key_ingredient'])))
    g.add((recipe, n.hasIngredient, ingredient))
    g.add((ingredient, n.hasQuantityOf, Literal(row['quantity'], datatype=XSD.integer)))
    g.add((ingredient, n.hasUnit, Literal(row['unit'], datatype=XSD.string)))

filename = 'output.ttl'
serialized_graph = g.serialize(format='turtle')
with open(filename, 'wb') as f:
    f.write(serialized_graph.encode('utf-8'))

SPARQL 
Visualization
